# Appliance Activations Matrix Populater for Use in Auto-encoder.

### Summary  
The purpose of this notebook is to convert the activations of appliances in the UKDALE and REFIT datasets into a binary matrix format which indicates time of the day active as well as date, day of the week and month. The output of this notebook will be used to feed an auto-encoder.

In [569]:
import pandas as pd
import numpy as np
import datetime as dt

In [570]:
#Integer input corresponding to the required house. Just makes cycling through easier.
i = 1
print('House', i)

House 1


In [571]:
#Input data needed.
#activations: csv of appliance activations over the course of measurement of the dataset. Taken from the nilm_analyzer.
activations = pd.read_csv(rf'...path...\nilm_analyzer_EM401\Working Files\Autoencoder\Autoencoder Data\Activations\R_h{i}_activations.csv', index_col=0)
#activations = pd.read_csv(rf'...path...\nilm_analyzer_EM401\Working Files\Autoencoder\Autoencoder Data\Activations\UK_h{i}_activations.csv', index_col=0)
#date_range: Any csv that houses the full date range of the dataset - in the case of UKDALE, mains files would suffice.
date_range = pd.read_csv(rf'...path...\nilm_analyzer_EM401\refit\House_{i}.csv', header = [0,1])
#date_range = pd.read_csv(rf'...path...\nilm_analyzer_EM401\Large Data\Reactive Power Mains Data\h{i}_mains.csv', header = [0,1])
date_range.head()

,Time,Unix,Aggregate,Appliance1,Appliance2,Appliance3,Appliance4,Appliance5,Appliance6,Appliance7,Appliance8,Appliance9
,2013-10-09 13:06:17,1381323977,523,74,0,69,0,0,0,0,0,1
0,2013-10-09 13:06:31,1381323991,526,75,0,69,0,0,0,0,0,1
1,2013-10-09 13:06:46,1381324006,540,74,0,68,0,0,0,0,0,1
2,2013-10-09 13:07:01,1381324021,532,74,0,68,0,0,0,0,0,1
3,2013-10-09 13:07:15,1381324035,540,74,0,69,0,0,0,0,0,1
4,2013-10-09 13:07:18,1381324038,539,74,0,69,0,0,0,0,0,1


In [572]:
#Check if correct. Should be date as a string 'yyyy-mm-dd HH:mm:ss'.
date_range.iloc[0,0]

'2013-10-09 13:06:31'

In [573]:
#Function that takes input activations and matrix dataframes and returns the final populated matrix dataframe.
def populate(matrix, activations):
    dfm = matrix
    dfa = activations
    starttime = []
    date = []
    endtime = []
    i = 0
    j = 0
    k = 0

    #Extracts start time for each activation as well as date of activation.
    while i < len(dfa):
        a = dfa.iloc[i]['activity_start']
        b = a.time()
        c = a.date()
        starttime.append(b)
        date.append(c)
        i += 1  

    #Same as above but for end times.
    while j < len(dfa):
        a = dfa.iloc[j]['activity_end']
        b = a.time()
        c = a.date()
        endtime.append(b)
        j += 1

    #Stores activations in seperate dataframes of type str.
    dfstarttime = pd.DataFrame(starttime).astype(str)
    dfendtime = pd.DataFrame(endtime).astype(str)
    dfdate = pd.DataFrame(date).astype(str)

    #Populates the matrix dataframe with binary 1's for each activation date, time and length.
    while k < len(dfa):
        dfm.loc[dfdate.iloc[k,0], dfstarttime.iloc[k,0]:dfendtime.iloc[k,0]] = 1
        k += 1
    
    return dfm
    

In [574]:
#Simple function for use in the matrix dataframe creation.
def get_month_str(month):
    monthnum = month.month
    if monthnum == 1:
        return str('January')
    elif monthnum == 2:
        return str('February')
    elif monthnum == 3:
        return str('March')
    elif monthnum == 4:
        return str('April')
    elif monthnum == 5:
        return str('May')
    elif monthnum == 6:
        return str('June')
    elif monthnum == 7:
        return str('July')
    elif monthnum == 8:
        return str('August')
    elif monthnum == 9:
        return str('September')
    elif monthnum == 10:
        return str('October')
    elif monthnum == 11:
        return str('November')
    elif monthnum == 12:
        return str('December')

In [575]:
def get_month(month):
    monthnum = month.month
    return monthnum

In [576]:
#Simple function for use in the matrix dataframe creation.
def get_DoW_str(day):
    daynum = day.dayofweek
    if daynum == 0:
        return str('Monday')
    elif daynum == 1:
        return str('Tuesday')
    elif daynum == 2:
        return str('Wednesday')
    elif daynum == 3:
        return str('Thursday')
    elif daynum == 4:
        return str('Friday')
    elif daynum == 5:
        return str('Saturday')
    elif daynum == 6:
        return str('Sunday')

In [577]:
def get_DoW(day):
    daynum = day.dayofweek
    return daynum

In [578]:
#Function to seperate time from a datetime object. May be unnecessary with x.time() command but works regardless.
def take_time(time):
    t = dt.datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
    x = t.strftime('%H:%M:%S')
    return x

In [579]:
#Formats the input activations dataframe.
activations = activations.sort_values(by='activity_start')
activations['activity_start'] = pd.to_datetime(activations['activity_start'])
activations['activity_end'] = pd.to_datetime(activations['activity_end'])
activations['activity_start'] = activations['activity_start'].dt.round('min')
activations['activity_end'] = activations['activity_end'].dt.round('min')

In [580]:
#Extracts the required first and last dates from the input date_range dataframe.
start = pd.to_datetime('{}'.format(date_range.iloc[0,0]))
end = pd.to_datetime('{}'.format(date_range.iloc[-1,0]))
start = start.date()
end = end.date()
print(start)
print(end)

2013-10-09
2015-07-10


In [581]:
#Index and Column set-up for the blank matrix dataframe.
index = [pd.date_range(start='{}'.format(start), end='{}'.format(end))]
columns = pd.DataFrame(pd.date_range('2001-4-15', periods=1440, freq='min')).astype(str)
columns = columns[0].apply(take_time)

In [582]:
#Dataframe 0's matrix with index being dates measured, columns being minutes in the day and a day of week and month column associated to each index date.
matrix = pd.DataFrame(0, index = index, columns = columns)
matrix.reset_index(names = 'index', inplace = True)
#matrix['day_of_week'] = matrix['index'].apply(get_DoW)
#matrix['month'] = matrix['index'].apply(get_month)
matrix['index'].astype(str)
matrix.set_index('index', inplace = True)


In [583]:
matrix.head()

,00:00:00,00:01:00,00:02:00,00:03:00,00:04:00,00:05:00,00:06:00,00:07:00,00:08:00,00:09:00,...,23:50:00,23:51:00,23:52:00,23:53:00,23:54:00,23:55:00,23:56:00,23:57:00,23:58:00,23:59:00
index,,,,,,,,,,,,,,,,,,,,,
2013-10-09,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-10-10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-10-11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-10-12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-10-13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [584]:
populate(matrix, activations)

,00:00:00,00:01:00,00:02:00,00:03:00,00:04:00,00:05:00,00:06:00,00:07:00,00:08:00,00:09:00,...,23:50:00,23:51:00,23:52:00,23:53:00,23:54:00,23:55:00,23:56:00,23:57:00,23:58:00,23:59:00
index,,,,,,,,,,,,,,,,,,,,,
2013-10-09,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-10-10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-10-11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-10-12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-10-13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-07-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-07-07,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-07-08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [585]:
#Filters the matrix down to 444 days of data so that each house will have equal dimensions.
matrix.drop(matrix.index[444:-1], inplace = True)
matrix.drop(matrix.index[-1], inplace = True)
matrix.shape


(444, 1440)

In [586]:
matrix.head()

,00:00:00,00:01:00,00:02:00,00:03:00,00:04:00,00:05:00,00:06:00,00:07:00,00:08:00,00:09:00,...,23:50:00,23:51:00,23:52:00,23:53:00,23:54:00,23:55:00,23:56:00,23:57:00,23:58:00,23:59:00
index,,,,,,,,,,,,,,,,,,,,,
2013-10-09,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-10-10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-10-11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-10-12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-10-13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [587]:
matrix.to_csv(rf'...path...\nilm_analyzer_EM401\Working Files\Autoencoder\Autoencoder Data\Populated Matrices\R_h{i}_DW.csv')